# ELAS — Cosmological Tensions (H0 & sigma8)
_Notebook auto-gen • 2025-10-19 13:45:56_

This notebook measures tensions using standardized datasets:
- Pantheon+ (SNe) -> background (Omega_m, H0)
- DESI DR2 (synthetic) BAO + covariance -> complementary
- Cosmic chronometers H(z) -> validation
- RSD f*sigma8(z) -> growth

Outputs: figures, JSON tables, and a Markdown report + ZIP.

In [2]:
# Cell 1 — Setup & paths
import os, json, math, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv

plt.rcParams["figure.dpi"] = 140

ROOT = "/content/ELAS/output"
TAB  = f"{ROOT}/tables"
FIG  = f"{ROOT}/figures"
REP  = f"{ROOT}/tensions_report"
os.makedirs(TAB, exist_ok=True)
os.makedirs(FIG, exist_ok=True)
os.makedirs(REP, exist_ok=True)

SN_FILE   = f"{TAB}/sn_pantheonplus_standardized.csv"
BAO_MEAS  = f"{TAB}/bao_desi_meas.csv"
BAO_COVNP = f"{TAB}/bao_desi_cov.npy"
FS8_FILE  = "/content/rsd_fs8.csv"
CC_FILE   = "/content/cc_Hz.csv"

for p in [SN_FILE, BAO_MEAS, BAO_COVNP]:
    assert os.path.exists(p), f"Missing: {p}"

print("OK paths:\n -", SN_FILE, "\n -", BAO_MEAS, "\n -", BAO_COVNP)


AssertionError: Missing: /content/ELAS/output/tables/sn_pantheonplus_standardized.csv

In [ ]:
# Cell 2 — Basic LCDM background
import numpy as np
c_kms = 299792.458

class Cosmo:
    def __init__(self, Om=0.32, H0=67.7):
        self.Om = float(Om)
        self.H0 = float(H0)

def E(z, p: Cosmo):
    z = np.asarray(z, float)
    return np.sqrt(p.Om*(1+z)**3 + (1-p.Om))

def DH(p: Cosmo):
    return c_kms/p.H0

def DM(z, p: Cosmo):
    z = float(z)
    if z == 0.0:
        return 0.0
    zz = np.linspace(0, z, 1201)
    return DH(p) * np.trapezoid(1.0/E(zz, p), zz)

def DL(z, p: Cosmo):
    return (1+z) * DM(z, p)

def mu_theory(z, p: Cosmo):
    if np.isscalar(z):
        return 5*np.log10(DL(float(z), p)) + 25
    return np.array([5*np.log10(DL(zi, p)) + 25 for zi in z], float)

def yth_bao(z, ob, p: Cosmo):
    if ob == "DM_OVER_RD":
        return DM(z, p)
    if ob == "DH_OVER_RD":
        return DH(p)/E(z, p)
    if ob == "DV_OVER_RD":
        return (DM(z, p)**2 * z * DH(p)/E(z, p))**(1/3)
    if ob == "RD_OVER_DV":
        val = (DM(z, p)**2 * z * DH(p)/E(z, p))**(1/3)
        return 1.0/max(val, 1e-12)
    val = (DM(z, p)**2 * z * DH(p)/E(z, p))**(1/3)
    return val

print("Cosmo functions ready.")


In [ ]:
# Cell 3 — Quick LCDM fit on SN + BAO (free amplitudes)
import pandas as pd, numpy as np, json
from numpy.linalg import inv

sn  = pd.read_csv(SN_FILE)
bao = pd.read_csv(BAO_MEAS)
cov = np.load(BAO_COVNP)

z_sn = sn["z"].to_numpy(float)
y_sn = sn["y"].to_numpy(float)
se_sn= sn["y_err"].to_numpy(float)
mu0  = sn["mu_th_LCDM"].to_numpy(float)

z_b  = bao["z_eff"].to_numpy(float)
y_b  = bao["value"].to_numpy(float)
obs  = bao["observable"].astype(str).str.upper().tolist()

Cinv_sn = inv(np.diag(se_sn**2) + 1e-10*np.eye(len(se_sn)))
Cinv_ba = inv(cov + 1e-10*np.eye(len(cov)))

def chi2_SN(p: Cosmo):
    mu_th = mu_theory(z_sn, p)
    q = float((mu_th.T@Cinv_sn@y_sn)/(mu_th.T@Cinv_sn@mu_th))
    r = y_sn - q*mu_th
    return float(r.T@Cinv_sn@r)

def chi2_BAO(p: Cosmo):
    y0 = np.array([yth_bao(zz, ob, p) for zz, ob in zip(z_b, obs)])
    q = float((y0.T@Cinv_ba@y_b)/(y0.T@Cinv_ba@y0))
    r = y_b - q*y0
    return float(r.T@Cinv_ba@r)

def chi2_tot(p: Cosmo):
    return chi2_SN(p) + chi2_BAO(p)

Om_grid = np.linspace(0.15, 0.5, 29)
H0_grid = np.linspace(60.0, 75.0, 31)
best = (1e99, Cosmo())
for Om in Om_grid:
    for H0 in H0_grid:
        c2 = chi2_tot(Cosmo(Om, H0))
        if c2 < best[0]:
            best = (c2, Cosmo(Om, H0))
c2_best, p_best = best
print({"chi2": c2_best, "Om": p_best.Om, "H0": p_best.H0})

with open(f"{TAB}/tensions_fit_background.json","w") as f:
    json.dump({"Om":p_best.Om, "H0":p_best.H0, "chi2":c2_best}, f, indent=2)


In [ ]:
# Cell 4 — H(z) validation with cosmic chronometers
import pandas as pd, numpy as np, matplotlib.pyplot as plt, json, os
cc = pd.read_csv(CC_FILE)
cc.columns = [c.strip().lower() for c in cc.columns]
kz = [c for c in cc.columns if c.startswith('z')][0]
kh = [c for c in cc.columns if c.startswith('h') and 'err' not in c][0]
ke = [c for c in cc.columns if 'err' in c or 'sigma' in c][0]
z_cc   = cc[kz].to_numpy(float)
H_cc   = cc[kh].to_numpy(float)
H_err  = cc[ke].to_numpy(float)

def H_theory(z, p: Cosmo):
    return p.H0 * E(z, p)

Hz_th = H_theory(z_cc, p_best)
chi2_cc = float(((H_cc - Hz_th)/H_err @ (H_cc - Hz_th)/H_err))
print({"chi2_CC": chi2_cc, "N": len(z_cc)})

plt.figure(figsize=(6,3.2))
plt.errorbar(z_cc, H_cc, yerr=H_err, fmt='o', ms=3, capsize=2, label='CC data')
z_plot = np.linspace(0, max(2.0, z_cc.max()*1.05), 300)
plt.plot(z_plot, H_theory(z_plot, p_best), label=f"LCDM fit (Om={p_best.Om:.3f}, H0={p_best.H0:.1f})")
plt.xlabel("z"); plt.ylabel("H(z) [km s^-1 Mpc^-1]"); plt.legend(); plt.tight_layout()
plt.savefig(f"{FIG}/hz_cc_validation.png"); plt.close()
print("Figure:", f"{FIG}/hz_cc_validation.png")

with open(f"{TAB}/tensions_hz_cc.json","w") as f:
    json.dump({"chi2_CC": chi2_cc, "N": int(len(z_cc))}, f, indent=2)


In [ ]:
# Cell 5 — Growth and f*sigma8(z)
import pandas as pd, numpy as np, matplotlib.pyplot as plt, json
fs = pd.read_csv(FS8_FILE)
fs.columns = [c.strip().lower() for c in fs.columns]
kz  = [c for c in fs.columns if c.startswith('z')][0]
kfs = [c for c in fs.columns if 'fs8' in c and 'err' not in c][0]
ke  = [c for c in fs.columns if ('err' in c or 'sigma' in c) and 'fs8' in c][0]

z_fs   = fs[kz].to_numpy(float)
fs8    = fs[kfs].to_numpy(float)
fs8err = fs[ke].to_numpy(float)

def Omega_m_a(a, p: Cosmo):
    return p.Om / (p.Om + (1-p.Om)*a**3)

def growth_D(a_array, p: Cosmo):
    a_array = np.asarray(a_array, float)
    x = np.log(a_array)
    order = np.argsort(x)
    x = x[order]; a = np.exp(x)
    def dlnH_dlnA(a):
        return -1.5 * Omega_m_a(a, p)
    N = len(a)
    D = np.zeros(N); F = np.zeros(N)
    D[0] = a[0]; F[0] = a[0]
    for i in range(N-1):
        dx = x[i+1]-x[i]
        D2 = - (2 + dlnH_dlnA(a[i]))*F[i] + 1.5*Omega_m_a(a[i], p)*D[i]
        F[i+1] = F[i] + D2*dx
        D[i+1] = D[i] + F[i]*dx
    Dout = np.zeros(N); Dout[order] = D
    return Dout

def fs8_theory(z, p: Cosmo, sigma8_0=0.8):
    a = 1.0/(1.0+np.asarray(z, float))
    a_grid = np.linspace(1e-3, 1.0, 2000)
    D_grid = growth_D(a_grid, p)
    D1 = float(D_grid[-1])
    Da = np.interp(a, a_grid, D_grid)
    xg = np.log(a_grid)
    dDdx = np.gradient(D_grid, xg)
    fa = np.interp(a, a_grid, dDdx)/np.maximum(Da, 1e-20)
    return fa * sigma8_0 * (Da/np.maximum(D1,1e-20))

pdict = json.load(open(f"{TAB}/tensions_fit_background.json"))
p = Cosmo(pdict["Om"], pdict["H0"])

sig_grid = np.linspace(0.5, 1.1, 121)
best = (1e99, 0.8)
for s8 in sig_grid:
    th = fs8_theory(z_fs, p, sigma8_0=s8)
    r  = (fs8 - th)/fs8err
    c2 = float(r@r)
    if c2 < best[0]: best = (c2, s8)
chi2_fs8, sigma8_0_hat = best
print({"sigma8_0_hat": sigma8_0_hat, "chi2_fs8": chi2_fs8, "N": int(len(z_fs))})

z_plot = np.linspace(0, max(2.0, z_fs.max()*1.05), 300)
plt.figure(figsize=(6,3.2))
plt.errorbar(z_fs, fs8, yerr=fs8err, fmt='o', ms=3, capsize=2, label='RSD f*sigma8 data')
plt.plot(z_plot, fs8_theory(z_plot, p, sigma8_0=sigma8_0_hat), label=f"LCDM fit sigma8(0)={sigma8_0_hat:.2f}")
plt.xlabel("z"); plt.ylabel("f*sigma8(z)"); plt.legend(); plt.tight_layout()
plt.savefig(f"{FIG}/fs8_growth_validation.png"); plt.close()
print("Figure:", f"{FIG}/fs8_growth_validation.png")

with open(f"{TAB}/tensions_fs8.json","w") as f:
    json.dump({"sigma8_0_hat": sigma8_0_hat, "chi2_fs8": chi2_fs8, "N": int(len(z_fs))}, f, indent=2)


In [ ]:
# Cell 6 — Summary & report
import math, json, shutil
from textwrap import dedent

PLANCK_H0, PLANCK_H0_ERR = 67.4, 0.5
SH0ES_H0,  SH0ES_H0_ERR  = 73.0, 1.0
PLANCK_SIG8, PLANCK_SIG8_ERR = 0.811, 0.006

bg = json.load(open(f"{TAB}/tensions_fit_background.json"))
g  = json.load(open(f"{TAB}/tensions_fs8.json"))

H0_hat = float(bg["H0"]); Om_hat = float(bg["Om"]) ;
sigma8_hat = float(g["sigma8_0_hat"]) ;

def sigma_diff(mu_a, sig_a, mu_b, sig_b):
    return abs(mu_a - mu_b)/math.sqrt(sig_a**2 + sig_b**2)

zH0_Pl = sigma_diff(H0_hat, 1.5, PLANCK_H0, PLANCK_H0_ERR)
zH0_SH = sigma_diff(H0_hat, 1.5, SH0ES_H0, SH0ES_H0_ERR)
zS8_Pl = sigma_diff(sigma8_hat, 0.05, PLANCK_SIG8, PLANCK_SIG8_ERR)

summary = {
  "fit_LCDM": {"Om": Om_hat, "H0": H0_hat},
  "sigma8_0": sigma8_hat,
  "tensions": {
    "H0_vs_Planck_sigma": zH0_Pl,
    "H0_vs_SH0ES_sigma": zH0_SH,
    "sigma8_vs_Planck_sigma": zS8_Pl
  }
}
print(json.dumps(summary, indent=2))

md = dedent(f"""
# ELAS — Tensions cosmologiques (H0 & sigma8)

## Résumé
- Fit LCDM (SN+BAO): Omega_m = {Om_hat:.3f}, H0 = {H0_hat:.2f} km/s/Mpc
- Growth (RSD): sigma8(0) = {sigma8_hat:.3f}

### Tensions (approx en sigma)
- H0 vs Planck: {zH0_Pl:.2f} sigma
- H0 vs SH0ES: {zH0_SH:.2f} sigma
- sigma8 vs Planck: {zS8_Pl:.2f} sigma

## Figures
- hz_cc_validation.png
- fs8_growth_validation.png
""")

MD_PATH = f"{REP}/ELAS_tensions_report.md"
JS_PATH = f"{REP}/ELAS_tensions_report.json"
with open(MD_PATH, "w") as f: f.write(md)
with open(JS_PATH, "w") as f: json.dump(summary, f, indent=2)

ZIP_PATH = shutil.make_archive(f"{REP}", "zip", REP)
print("\n==== REPORT READY ====")
print("Markdown:", MD_PATH)
print("JSON    :", JS_PATH)
print("ZIP     :", ZIP_PATH)
